In [1]:
# N-gram tfidf 
# have not drop stopword 
# have dropping digits
"""
EQB = [ [EQB item] * number of EQB item] 
[EQB item] = [ [paragraph1 in ITB1] , [ paragraph2 in ITB2] , [paragraph3 in ITB3] ] 
"""

#only use EOEG,NC6,JGS,liwa as training data
#use PTT_LNG as valid data 



import pandas as pd
import numpy as np
import nltk
import math
import re

In [2]:
pd_EOEG = pd.read_excel('EQB.xlsx',sheet_name = 'EOEG' );
pd_NC6 = pd.read_excel("EQB.xlsx",sheet_name='NC6');
pd_JGS = pd.read_excel("EQB.xlsx",sheet_name= 'JGS' );
pd_liwa = pd.read_excel("EQB.xlsx",sheet_name= 'LIWA' );
pd_PTT = pd.read_excel("EQB.xlsx",sheet_name= 'PTT LNG' );

In [3]:
EQB = [ [] for i in range(406) ];

def add_EQB(df):
    check = df['paragraph'].isna();
    for i in range(len(df)): 
        if check[i] == False:
            EQB[i].append(df['paragraph'][i])

add_EQB(pd_NC6);
add_EQB(pd_EOEG);
add_EQB(pd_JGS);
add_EQB(pd_liwa);

fuck = pd_NC6['Quality Activity'];

In [4]:
from nltk.corpus import stopwords

from nltk.stem import SnowballStemmer
import  nltk;
import math;
from nltk.stem import PorterStemmer
stopword_list = list(set(stopwords.words('english')));
stopword_list.append(',');
stopword_list.append(')');
stopword_list.append('(');
stopword_list.append('.');
stopword_list.append('-');

stemmer = SnowballStemmer('english')


In [5]:
def getgram(paragraph , gram):
    ret = []
    for i in range( len(paragraph) - gram  ):
        combine = '';
        for j in range(gram):
            combine += paragraph[i+j] + ' '
        ret.append(combine);
    return ret;

test = ['test1' , 'test2' , 'test3' , '11' ,'22' , '33']
print(getgram(test,3))

['test1 test2 test3 ', 'test2 test3 11 ', 'test3 11 22 ']


In [6]:
def preprocess_paragraph(paragraph):
    raw_words = nltk.word_tokenize(paragraph); #tokenize words by nltk
    words = [stemmer.stem(w) for w in raw_words] # stem
    drop_digit = [] #drop digit
    for w in words:
        if re.findall(r'^[^\d]\w+',w):
            after = re.findall(r'^\w+$',w);
            if len(after) > 0:
                drop_digit.append(after[0])
            
    return drop_digit;

In [7]:
def update_dic(key , dic): #dictnary中令一个key的value+1
    if key not in dic:
        dic[key] = 1;
    else: 
        dic[key] += 1;
    return dic

In [8]:
def get_tf(EQB_field , df , gram):
    num_words = 0;
    freq = {};
    for paragraph in EQB_field:
        words = preprocess_paragraph(paragraph) #just drop 標點符號和數字
        words = getgram(words,gram) # get N-gram words
        num_words += len(words) # calculate the total number of these paragraphs
        for w in words:
            freq = update_dic( key = w , dic = freq) # calculate tf
    tf = {}        
    for (word , count) in freq.items():
        tf[word] = count * 1.0 / num_words; # 修正每个词为正确的频率
        df = update_dic( key = word , dic = df)    #calculate df
    return tf , df;

In [9]:
def get_tfidf(tf_list , df , N):
    tfidf_list = []
    for tf in tf_list:
        tfidf = {}
        for (word , tf_value) in tf.items():
            tfidf[word] = tf_value * math.log( N / ( 1 + df[word]) , 2 ) 
        tfidf_list.append(tfidf);
    return tfidf_list

In [10]:
def solve(EQB , gram):
    df = {}
    tf_list = []
    id_list = []
    N = len(EQB);
    
    item = pd_EOEG['Item'][0] 
    for i in range(len(EQB)):
        EQB_field = EQB[i];
        id = pd_EOEG['Quality Activity'][i] 
        if (pd_EOEG['Item'].isna()[i] == False):
            item = pd_EOEG['Item'][i] 
        
        if len(EQB_field) == 0:
            N = N - 1
            continue    
        tf , df = get_tf(EQB_field, df , gram)
        tf_list.append(tf)
        id_list.append(item + ' ' + id)
    tfidf_list = get_tfidf(tf_list , df , N)
    
    
    for i in range(len(tfidf_list)):
        tfidf = tfidf_list[i]
        id = id_list[i]
        print(id)
        s = pd.Series(tfidf)
        print(s.sort_values(ascending=False).head(10))


In [11]:
solve(EQB , gram = 2)

1.1 a. Minimun Vertical Clearance for Access
walkway and           0.165400
clearanc headroom     0.165400
headroom over         0.165400
platform walkway      0.165400
and elev              0.090713
elev platform         0.090713
pipe the              0.090713
overhead pipe         0.090713
for overhead          0.090713
and work              0.090713
dtype: float64
1.1 b. Minimun width Clearance for Access
width of             0.283217
area oper            0.194766
mainten area         0.194766
minimum platform     0.194766
platform minimum     0.194766
clearanc width       0.194766
of walkway           0.194766
walkway at           0.194766
platform pipe        0.194766
and it               0.194766
dtype: float64
1.2 a. Maximum distance between landings on a sectional ladder
platform shall      0.182831
for ladder          0.161513
space between       0.107676
meter abov          0.107676
vertic distanc      0.107676
offset platform     0.107676
between offset      0.107676
distanc

dtype: float64
1.2.1 Manway
fire water           0.198546
measur from          0.176671
main all             0.176671
main line            0.176671
be buri              0.176671
buri underground     0.176671
underground to       0.176671
to depth             0.176671
depth of             0.176671
least measur         0.176671
dtype: float64
1.2.2 Light vehicle way(<10 tons)
fire water           0.198546
measur from          0.176671
main all             0.176671
main line            0.176671
be buri              0.176671
buri underground     0.176671
underground to       0.176671
to depth             0.176671
depth of             0.176671
least measur         0.176671
dtype: float64
1.2.3 Vehicle way (10~20 tons)
fire water           0.198546
measur from          0.176671
main all             0.176671
main line            0.176671
be buri              0.176671
buri underground     0.176671
underground to       0.176671
to depth             0.176671
depth of             0.176671
least m

In [12]:
solve(EQB , gram = 3)

1.1 a. Minimun Vertical Clearance for Access
clearanc headroom over     0.174988
headroom over elev         0.095972
pipe the follow            0.095972
overhead pipe the          0.095972
for overhead pipe          0.095972
passageway and work        0.095972
abov walkway and           0.095972
work area and              0.095972
walkway work area          0.095972
platform walkway work      0.095972
dtype: float64
1.1 b. Minimun width Clearance for Access
mainten area oper            0.213615
and mainten area             0.213615
minimum platform minimum     0.213615
platform minimum width       0.213615
tabl clearanc width          0.213615
clearanc width of            0.213615
width of walkway             0.213615
of walkway at                0.213615
walkway at grade             0.213615
at grade or                  0.213615
dtype: float64
1.2 a. Maximum distance between landings on a sectional ladder
platform shall be           0.141732
space between offset        0.111295
ladder

dtype: float64
2.7 Wind Speed, annual average
and relat humid              0.407702
blower and air               0.203851
wet bulb temperatur          0.203851
condit temperatur averag     0.203851
temperatur averag daili      0.203851
averag daili temperatur      0.203851
daili temperatur design      0.203851
temperatur design condit     0.203851
design condit max            0.203851
condit max min               0.203851
dtype: float64
2.8.1 Ambient temperature, annual average
and relat humid              0.407702
blower and air               0.203851
wet bulb temperatur          0.203851
condit temperatur averag     0.203851
temperatur averag daili      0.203851
averag daili temperatur      0.203851
daili temperatur design      0.203851
temperatur design condit     0.203851
design condit max            0.203851
condit max min               0.203851
dtype: float64
2.8.2 Wind Speed, annual average
per nscp vol           0.509389
structur per nscp      0.509389
other structur per     0.